In [1]:
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np


daily_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/BI Argentina/Daily Táctico'

repos_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Bajadas_repo_ventas'
va05_direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Bajadas_repo_ventas'

va05_archivos = [x for x in os.listdir(repos_direc) if len(x)==32]

repos_archivos = [x for x in os.listdir(repos_direc) if len(x)==44][-12:]

#descargo el archivo de materiales
#materiales = pd.read_csv(daily_direc + '/Materiales2.csv', encoding='latin-1', sep = ';')
materiales = pd.read_excel(io = "C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/Tbl_maestro_articulos.xlsx", usecols="D:J,L")

#descargo el archivo de catalogo clientes
catalogo_clientes = pd.read_excel(io = "C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/TBL_canales_SAP_y_gerencias_ventas vR2D2.xlsx")
catalogo_clientes = catalogo_clientes[['Nbre_solicitante', 'Nbre_solicitante2',
       'Canal_distribucion', 'Tipo_estructura5', 'Gerente', 'KAM']]
catalogo_clientes.rename(columns={"Tipo_estructura5": "Canal"}, inplace = True)

#descargo el archivo de ship to x clientes
ship = pd.read_excel('C:/Users/rvozzi/OneDrive - genommalabinternacional/Catalogos/20221018 Detalle de Ship tos X Clientes ARG.XLSX', usecols='AN,AT')
ship.columns = ['ship to', 'ship to poblacion']
ship = ship.drop_duplicates()

#ver en el OI a que columna corresponde este mes
col_excel_u = 'DQ'

In [2]:
%%time

#proceso los archivos de pedidos

pedidos = pd.DataFrame()

for file in va05_archivos:
    
    z = pd.read_excel(va05_direc + '/' + file, usecols='A:Q')  
    
    #en el df pedidos voy acumulando todos los archivos
    pedidos = pd.concat([pedidos, z], ignore_index=True)
    
# agrupo segun el codigo material y documento de ventas
pedidos['Referencia de cliente (cabecera)'] = pedidos['Referencia de cliente (cabecera)'].fillna('-')
pedidos['Descripción Datos básicos B'] = pedidos['Descripción Datos básicos B'].fillna('-')

dict_replace = {"ZNCG" : "NC",
                "ZNCS" : "NC",
                "ZNCU" : "NC",
                "ZNDS" : "NC",
                "ZNSP" : "NC",
                "ZCOL" : "Fact",
                "ZPEA" : "Fact",
                "ZPRO" : "Fact",
                "ZPSA" : "Fact",
                "ZTEM" : "Fact"}

pedidos = pedidos.replace({"Clase doc.ventas": dict_replace})

pedidos = pedidos[pedidos['Clase doc.ventas'].isin(['NC', 'Fact'])]

pedidos = pedidos.groupby(['Fecha documento', 'Solicitante', 'Nombre del solicitante', 'Documento de ventas', 
            'Material', 'Descripción del material', 'Un.medida venta', 'Interlocutor', 'Creado por',
            'Referencia de cliente (cabecera)', 'Descripción Datos básicos B', 'Precio neto', 'Clase doc.ventas'])['Cantidad de pedido (Posición)','Valor neto (posición)'].sum()

pedidos = pedidos.reset_index()

#cambiar nombres de las columnas
pedidos_rename = ['Fe_documento', 'Nro_solicitante', 'Nbre_solicitante',
        'Doc_ventas', 'Cod_material', 'Descripcion',
        'Un.medida venta', 'Interlocutor', 'Creado por', 'Referencia_cliente', 'Descripción Datos básicos B',
        'precio_pedido', 'Tipo_doc', 'Cantidad_pedida', 'Valor_neto_pedido']

pedidos.rename(dict(zip(pedidos.columns,pedidos_rename)), axis=1, inplace=True)

pedidos['Doc_ventas'] = pedidos['Doc_ventas'].astype('int64')

C:\Users\rvozzi\anaconda3\lib\site-packages\pandas\io\excel\_base.py:1272: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  return self._reader.parse(
<timed exec>:31: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


Wall time: 7min 37s


In [3]:
materiales.columns

Index(['Cod_Material', 'Material', 'Empresa', 'Status', 'BU', 'Categoría',
       'Marca', 'Familia'],
      dtype='object')

In [4]:
#Son los materiales que estan en repo ventas que no estan en el maestro. El vector deberia tener un len de cero
pd.unique(pedidos[~pedidos['Cod_material'].isin(materiales['Cod_Material'])]['Cod_material']).astype('int64')

array([8110000881], dtype=int64)

In [5]:
pedidos

,Fe_documento,Nro_solicitante,Nbre_solicitante,Doc_ventas,Cod_material,Descripcion,Un.medida venta,Interlocutor,Creado por,Referencia_cliente,Descripción Datos básicos B,precio_pedido,Tipo_doc,Cantidad_pedida,Valor_neto_pedido
0,2022-09-01,1100311.0,FARMCITY S.A.,170153962,8.110000e+09,CICA ROLL ON OJOS X 15ML,PZA,1897.0,ARINTSF,1193479,-,1169.15,Fact,40,46766.01
1,2022-09-01,1100311.0,FARMCITY S.A.,170153962,8.110000e+09,CICA BEAUTY CARE X 50GGLA,PZA,1897.0,ARINTSF,1193479,-,1202.77,Fact,60,72166.46
2,2022-09-01,1100311.0,FARMCITY S.A.,170153962,8.110000e+09,CICATRICURE SERUM ANTIMANCHAS 3.4G,PZA,1897.0,ARINTSF,1193479,-,1768.64,Fact,12,21223.65
3,2022-09-01,1100311.0,FARMCITY S.A.,170153962,8.110000e+09,CICATRICURE GEL X 60 G NF,PZA,1897.0,ARINTSF,1193479,-,933.75,Fact,32,29879.88
4,2022-09-01,1100311.0,FARMCITY S.A.,170153962,8.110001e+09,TEATRICAL CREMA ACLARANTE X 200G,PZA,1897.0,ARINTSF,1193479,-,465.70,Fact,12,5588.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148186,2023-01-18,1101205.0,AZUL AGRUPACION DE COLABORACION,170166101,8.110001e+09,DURACELL PILA LITIO 2025 X1 TIRA,PZA,2491.0,ARINTSF,170123,-,106.51,Fact,200,21301.20
148187,2023-01-18,1101205.0,AZUL AGRUPACION DE COLABORACION,170166101,8.110001e+09,DURACELL PILA LITIO 2016 X1 TIRA,PZA,2491.0,ARINTSF,170123,-,106.51,Fact,200,21301.20
148188,2023-01-18,1101228.0,SAJEL SA,170166175,9.100001e+09,DURACELL N/A GASTOS PUBLICIDAD,SER,1101228.0,DCALCAGNO,ND NCG 87407,-,166403.22,NC,0,166403.22
148189,2023-01-18,1101228.0,SAJEL SA,170166194,9.100000e+09,N/A GASTOS PUBLICIDAD,SER,1101228.0,DCALCAGNO,NCG DIC22 FC X 21474,NCG DIC22 FC X 21474,166403.22,NC,1,166403.22


In [6]:
%%time

#proceso todos los archivos de facturacion

#me creo un df vacio donde voy acumulando todo
repos_ventas = pd.DataFrame()

#nombres de las columnas del archivo de SAP
SAP_names = ['Doc.fact.', 'Folio Entrega', 'Documento de ventas', 'Referencia cliente', 'Destinatario', 'Material',
             'Cantidad facturada', 'Valor neto', 'Fecha factura','Organización ventas']

#nombres de las columnas de BD_Venta
BD_names = ['Nro_factura', 'Nro_entrega', 'Nro_pedido', 'Nro_OC', 'Nro_Destinatario', 'Cod_Material',
            'Cantidad_facturada', 'Valor_neto_facturado', 'Fe_factura', 'Organizacion ventas', 'Año_Mes']

#hago un for que me vaya agregando todo hasta mayo
for file in repos_archivos:
    
    #Leo cada archivo y filtro las columnas
    x = pd.read_excel(repos_direc + '/' + file)    
    x = x[SAP_names]
    
    #Agrego la columna Año_Mes usando el nombre del archivo
    x['Año_Mes'] = file[-12:-5].replace('_','-')
    
    #Filtro filas, El material 9100000322 lo pongo cuando tenga los montos
    x = x[x['Organización ventas']=='VNAR']
    x = x[~x['Material'].isin([9100000322, 9100000316])]
    
    #elimino la columna 'Organización ventas' ya que no me sirve mas
    x.drop('Organización ventas', axis=1, inplace=True)
    
    #en el df repos_ventas voy acumulando todos los archivos
    repos_ventas = pd.concat([repos_ventas, x], ignore_index=True)
    
    
repos_ventas.rename(dict(zip(SAP_names,BD_names)), axis=1, inplace=True)

repos_ventas['Cantidad_facturada'] = repos_ventas['Cantidad_facturada'].astype('int64')

repos_ventas['Nro_OC'] = repos_ventas['Nro_OC'].fillna('-')

Wall time: 8min 10s


In [7]:
file

'Reporte de Ventas Nacional vNVA 2023_01.XLSX'

In [8]:
repos_ventas = repos_ventas.groupby(['Nro_pedido', 'Cod_Material', 'Nro_Destinatario', 'Fe_factura', 'Nro_OC',
                                      'Nro_factura', 'Nro_entrega', 'Año_Mes'])['Cantidad_facturada', 'Valor_neto_facturado'].sum()

repos_ventas = repos_ventas.reset_index()

C:\Users\rvozzi\AppData\Local\Temp/ipykernel_24212/4246101649.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  repos_ventas = repos_ventas.groupby(['Nro_pedido', 'Cod_Material', 'Nro_Destinatario', 'Fe_factura', 'Nro_OC',


In [9]:
def read_oi(col_excel, col_name):
    
    oi = pd.read_excel(daily_direc + '/OI_CANAL/OI 2022-12 006 CERRADO.XLSX', header = 6, sheet_name = 'Rolling Forecast', 
                   usecols = 'L,' + col_excel)
    
    oi.drop(oi.tail(4).index,inplace=True) # drop last 4 rows

    #agrego la columna de tipo 
    oi['Tipo'] = 'proy'

    #agrego Año_Mes
    oi['Año_Mes'] = datetime.today().strftime('%Y') + '-' + datetime.today().strftime('%m')

    #renombro columnas
    oi_names = ['Cod_Material', col_name, 'Tipo', 'Año_Mes']
    oi.rename(dict(zip(oi.columns.to_list(),oi_names)), axis=1, inplace=True)

    #convierto unidades a int
    oi[col_name] = oi[col_name].fillna(0)
    oi[col_name] = oi[col_name].astype('int64')
    oi = oi.dropna(subset=['Cod_Material'])
    oi['Cod_Material'] = oi['Cod_Material'].astype('int64')

    return oi

In [10]:
oi = read_oi(col_excel_u, 'Unidades')
oi.drop_duplicates(inplace = True)
vec = oi[oi['Unidades']>0]['Cod_Material'].to_numpy()

C:\Users\rvozzi\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Sparkline Group extension is not supported and will be removed
  warn(msg)


In [11]:
#hago un pedidos left join repos_ventas
data = pd.merge(pedidos,
                repos_ventas,
                left_on = ['Doc_ventas', 'Cod_material'],
                right_on = ['Nro_pedido', 'Cod_Material'],
                how = 'left')
data.drop('Cod_Material', axis=1, inplace=True)

#transformo las cantidades y montos pedidos a Negativo de las notas de credito
data.loc[data['Tipo_doc']=='NC', 'Valor_neto_pedido'] = -data['Valor_neto_pedido']
data.loc[data['Tipo_doc']=='NC', 'Cantidad_pedida'] = -data['Cantidad_pedida']

#transformo las unidades facturadas y pedidas de las notas de credito que tienen nro_entrega = 170 a cero
data.loc[(data['Tipo_doc']=='NC') & (data['Nro_entrega'].astype(str).str[:3]=='170'), 'Cantidad_facturada'] = 0
data.loc[(data['Tipo_doc']=='NC') & (data['Nro_entrega'].astype(str).str[:3]=='170'), 'Cantidad_pedida'] = 0

#calculo los valores de corte
data['Cantidad_facturada'] = data['Cantidad_facturada'].fillna(0)
data['Cantidad_corte'] = data['Cantidad_pedida'] - data['Cantidad_facturada']
data['Valor_neto_corte'] = data['precio_pedido']*data['Cantidad_corte']

#cambio las fecha de factura que son nulas por fecha de pedido
data['Fecha_filtro'] = data['Fe_factura']
data.loc[data['Fecha_filtro'].isnull(), 'Fecha_filtro'] = data['Fe_documento']

data['Cod_material'] = data['Cod_material'].astype('int64')

#Hago un merge de la tabla con la tabla de materiales
data = pd.merge(data,
                materiales,
                left_on = 'Cod_material',
                right_on = 'Cod_Material',
                how = 'left')

#Hago un merge de la tabla con el catalogo de clientes
data = pd.merge(data,
                catalogo_clientes,
                left_on = 'Nbre_solicitante',
                right_on = 'Nbre_solicitante',
                how = 'left')

data.loc[data['Nbre_solicitante2'] != 'OTROS', 'Nbre_solicitante'] = data['Nbre_solicitante2']
data.drop(['Nbre_solicitante2'], axis=1, inplace=True)

#Hago un merge de la tabla con el archivo de detalle ship to x clientes
data = pd.merge(data,
                ship,
                left_on = 'Interlocutor',
                right_on = 'ship to',
                how = 'left')

#Me creo una columna que si el sku esta forecasteado (Un>0) vale 1, sino 0
data['Forecast'] = data['Cod_material'].isin(vec).astype(int)

In [12]:
data1 = data[data['Fecha_filtro'].dt.strftime('%Y-%m') == '2022-10']
#pedidos1 = pedidos[pedidos['Fe_documento'].dt.strftime('%Y-%m') == '2022-10']
data1 = data1[~data1['Descripción Datos básicos B'].isin(['RECUPERO'])]

In [13]:
data1['Cantidad_facturada'].sum()/data1['Cantidad_pedida'].sum()

0.8232815433496216

In [14]:
data1['Cantidad_facturada'].sum()

4874515.0

In [15]:
data1['Cantidad_pedida'].sum()

5920836

In [16]:
data.to_csv(path_or_buf = daily_direc + '/base_va05.csv',
                    sep = ',', index = False)

In [17]:
data.columns

Index(['Fe_documento', 'Nro_solicitante', 'Nbre_solicitante', 'Doc_ventas',
       'Cod_material', 'Descripcion', 'Un.medida venta', 'Interlocutor',
       'Creado por', 'Referencia_cliente', 'Descripción Datos básicos B',
       'precio_pedido', 'Tipo_doc', 'Cantidad_pedida', 'Valor_neto_pedido',
       'Nro_pedido', 'Nro_Destinatario', 'Fe_factura', 'Nro_OC', 'Nro_factura',
       'Nro_entrega', 'Año_Mes', 'Cantidad_facturada', 'Valor_neto_facturado',
       'Cantidad_corte', 'Valor_neto_corte', 'Fecha_filtro', 'Cod_Material',
       'Material', 'Empresa', 'Status', 'BU', 'Categoría', 'Marca', 'Familia',
       'Canal_distribucion', 'Canal', 'Gerente', 'KAM', 'ship to',
       'ship to poblacion', 'Forecast'],
      dtype='object')

In [18]:
data

,Fe_documento,Nro_solicitante,Nbre_solicitante,Doc_ventas,Cod_material,Descripcion,Un.medida venta,Interlocutor,Creado por,Referencia_cliente,...,Categoría,Marca,Familia,Canal_distribucion,Canal,Gerente,KAM,ship to,ship to poblacion,Forecast
0,2022-09-01,1100311.0,FARMACITY,170153962,8110000071,CICA ROLL ON OJOS X 15ML,PZA,1897.0,ARINTSF,1193479,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ROLL ON OJOS,FN,2.Cadenas farmacias,Catalina Sabogal,Carolina Kronemberger,1897.0,DON TORCUATO,1
1,2022-09-01,1100311.0,FARMACITY,170153962,8110000072,CICA BEAUTY CARE X 50GGLA,PZA,1897.0,ARINTSF,1193479,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE BEAUTY CARE,FN,2.Cadenas farmacias,Catalina Sabogal,Carolina Kronemberger,1897.0,DON TORCUATO,1
2,2022-09-01,1100311.0,FARMACITY,170153962,8110000293,CICATRICURE SERUM ANTIMANCHAS 3.4G,PZA,1897.0,ARINTSF,1193479,...,SKIN CARE,CICATRICURE FACIAL,CICATRICURE ANTIMANCHAS,FN,2.Cadenas farmacias,Catalina Sabogal,Carolina Kronemberger,1897.0,DON TORCUATO,0
3,2022-09-01,1100311.0,FARMACITY,170153962,8110000398,CICATRICURE GEL X 60 G NF,PZA,1897.0,ARINTSF,1193479,...,SKIN CARE,CICATRICURE GEL,CICATRICURE GEL,FN,2.Cadenas farmacias,Catalina Sabogal,Carolina Kronemberger,1897.0,DON TORCUATO,1
4,2022-09-01,1100311.0,FARMACITY,170153962,8110000560,TEATRICAL CREMA ACLARANTE X 200G,PZA,1897.0,ARINTSF,1193479,...,SKIN CARE,TEATRICAL,TEATRICAL,FN,2.Cadenas farmacias,Catalina Sabogal,Carolina Kronemberger,1897.0,DON TORCUATO,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148193,2023-01-18,1101205.0,FARMAPLUS,170166101,8110000887,DURACELL PILA LITIO 2025 X1 TIRA,PZA,2491.0,ARINTSF,170123,...,BATTERIES,DURACELL,Pilas litio,FR,6.Ecommerce,Sandra Borggeto,Sandra Borggeto,2491.0,CABA,1
148194,2023-01-18,1101205.0,FARMAPLUS,170166101,8110000888,DURACELL PILA LITIO 2016 X1 TIRA,PZA,2491.0,ARINTSF,170123,...,BATTERIES,DURACELL,Pilas litio,FR,6.Ecommerce,Sandra Borggeto,Sandra Borggeto,2491.0,CABA,1
148195,2023-01-18,1101228.0,SUPER RECONQUISTA,170166175,9100001459,DURACELL N/A GASTOS PUBLICIDAD,SER,1101228.0,DCALCAGNO,ND NCG 87407,...,BATTERIES,DURACELL,NaN,A2,3.Supermercados,Vacante,Alejandro Caramutti,NaN,NaN,0
148196,2023-01-18,1101228.0,SUPER RECONQUISTA,170166194,9100000319,N/A GASTOS PUBLICIDAD,SER,1101228.0,DCALCAGNO,NCG DIC22 FC X 21474,...,NaN,NaN,NaN,A2,3.Supermercados,Vacante,Alejandro Caramutti,NaN,NaN,0
